In [1]:
import rasterio
import geopandas as gpd
import pandas as pd
import numpy as np
from rasterio.mask import mask

In [2]:
data_path = '../../Initial Phase/int_out/comparison.gpkg'
ndvi_path = '../../Initial Phase/2018_NDVI_LVG_tile.tif'

In [3]:
with rasterio.open(ndvi_path) as src:
    ndvi_crs = src.crs

In [4]:
layers = {
    "common_area": gpd.read_file(data_path, layer='common_areas'),
    "only_deter": gpd.read_file(data_path, layer='only_deter'),
    "only_icv": gpd.read_file(data_path, layer='only_icv'),
}

for name, gdf in layers.items():
    layers[name] = gdf.to_crs(ndvi_crs)
    
    layers[name]["geometry"] = layers[name]["geometry"].buffer(0)

In [5]:
mean_values = {}

# Open raster once for efficiency
with rasterio.open(ndvi_path) as src:
    for name, gdf in layers.items():
        # Combine geometries of each layer
        geometry = [gdf.geometry.union_all()]
        
        # Mask the raster using the combined geometry
        masked, _ = mask(src, geometry, crop=True)
        masked = masked.astype(float)
        
        # Replace NoData with NaN and calculate mean
        masked[masked == src.nodata] = np.nan
        mean_values[name] = np.nanmean(masked)

for layer, mean in mean_values.items():
    print(f"Average NDVI for {layer}: {mean:.4f}")

Average NDVI for common_area: 0.8377
Average NDVI for only_deter: 0.8197
Average NDVI for only_icv: 0.8511


In [6]:
layers = {
    "common_area": gpd.read_file(data_path, layer='common_areas'),
    "only_deter": gpd.read_file(data_path, layer='only_deter'),
    "only_icv": gpd.read_file(data_path, layer='only_icv')
}

year_columns = {
    "common_area": "YEAR",
    "only_deter": "YEAR",
    "only_icv": "Ano"
}

for name, gdf in layers.items():
    layers[name] = gdf.to_crs(ndvi_crs)
    layers[name][year_columns[name]] = pd.to_numeric(layers[name][year_columns[name]], errors="coerce")
    layers[name] = layers[name][layers[name][year_columns[name]] == 2018]
    layers[name]["geometry"] = layers[name]["geometry"].buffer(0)

In [7]:
mean_values = {}

# Open raster once for efficiency
with rasterio.open(ndvi_path) as src:
    for name, gdf in layers.items():
        # Combine geometries of each layer
        geometry = [gdf.geometry.union_all()]
        
        # Mask the raster using the combined geometry
        masked, _ = mask(src, geometry, crop=True)
        masked = masked.astype(float)
        
        # Replace NoData with NaN and calculate mean
        masked[masked == src.nodata] = np.nan
        mean_values[name] = np.nanmean(masked)

for layer, mean in mean_values.items():
    print(f"Average NDVI for {layer}: {mean:.4f}")

Average NDVI for common_area: 0.8405
Average NDVI for only_deter: 0.7741
Average NDVI for only_icv: 0.8330
